In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# DOMAIN = 'semparse_cdr'
# DOMAIN = 'semparse_spouse'
DOMAIN = 'semparse_test'

In [3]:
import os

if DOMAIN in ['semparse_test', 'semparse_spouse']:
    os.environ['SNORKELDB'] = 'postgres:///semparse_spouse'
else:
    os.environ['SNORKELDB'] = 'postgres:///semparse_cdr'

In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

In [5]:
from snorkel.models import candidate_subclass

if DOMAIN in ['semparse_test', 'semparse_spouse']:
    Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
    candidate_class = Spouse
elif DOMAIN == 'semparse_cdr':
    ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
    candidate_class = ChemicalDisease
else:
    raise Exception("Invalid DOMAIN.")

In [6]:
candidates = session.query(candidate_class).all()
print("Candidates: {}".format(len(candidates)))

Candidates: 543


In [7]:
%%time

if DOMAIN == 'semparse_test':
    user_lists = {
        'colors':['red','green','blue'],
        'bluebird':['blue','bird','fly'],
        'greek':['alpha','beta','gamma'],
        'letters':['a','B','C'],
        'smalls':['a','b','c','d'],
        'spouse':['wife','husband','spouse']}
elif DOMAIN == 'spouse':
    user_lists = {
        'spouse':['wife','husband','spouse'],
        'family':['father', 'mother', 'brother', 'sister']}
elif DOMAIN == 'semparse_cdr':
    import bz2
    import cPickle
    if 'ctd_unspecified' not in globals():
        with bz2.BZ2File('data/ctd.pkl.bz2', 'rb') as ctd_f:
            ctd_unspecified, ctd_therapy, ctd_marker = cPickle.load(ctd_f)
    user_lists = {
        'uncertain': ['combin', 'possible', 'unlikely'],
        'causal': ['causes', 'caused', 'induce', 'induces', 'induced', 'associated with'],
        'treat': ['treat', 'effective', 'prevent', 'resistant', 'slow', 'promise', 'therap'],
        'procedure': ['inject', 'administrat'],
        'patient': ['in a patient with', 'in patients with'],
        'weak': ['none', 'although', 'was carried out', 'was conducted', 'seems', 
                 'suggests', 'risk', 'implicated', 'the aim', 'to investigate',
                 'to assess', 'to study'],
        'ctd_unspecified': ctd_unspecified,
        'ctd_therapy': ctd_therapy,
        'ctd_marker': ctd_marker,
    }

CPU times: user 7 µs, sys: 3 µs, total: 10 µs
Wall time: 12.9 µs


In [8]:
from semparse_examples import get_examples

examples = get_examples(DOMAIN, candidates)

In [9]:
from snorkel.semantic import (get_left_phrases, get_right_phrases, 
                              get_between_phrases, get_sentence_phrases)
c = candidates[1]
print c[0].get_parent()._asdict()['text']
print c

\n \nThe Duke was given a large cardboard hand to fill in and named his immediate family - better known as wife Kate, Prince Harry, the Prince of Wales, the Queen and the Duke of Edinburgh - before signing the palm with his name. 
Spouse(Span("Kate", sentence=2686, chars=[112,115], words=[24,24]), Span("Harry", sentence=2686, chars=[125,129], words=[27,27]))


In [10]:
# print c[0].get_attrib_tokens(a='words')
# c[0].get_attrib_tokens(a='entity_cids')

In [11]:
# for p in get_left_phrases(c[0], cmp='.leq', num=3, n_max=3):
#     print getattr(p, 'text')

In [12]:
# c[0].get_parent()._asdict()

In [13]:
# from snorkel.viewer import SentenceNgramViewer
# sv = SentenceNgramViewer(candidates[:300], session)
# sv

In [14]:
# c = sv.get_selected()
# print c
# print c[0].get_parent()._asdict()['pos_tags']
# print c[0].get_parent()._asdict()['ner_tags']

In [15]:
# hash(c)

In [16]:
# examples[0].display()

In [17]:
# examples[28].candidate[0].get_parent()

In [18]:
from snorkel.semantic import SemanticParser

%time sp = SemanticParser(candidate_class, user_lists, absorb=False)

Created grammar with 297 rules
CPU times: user 3.18 ms, sys: 5.55 ms, total: 8.73 ms
Wall time: 9.73 ms


In [19]:
%time results = sp.evaluate(examples,\
                            show_everything=False,\
                            show_explanation=False,\
                            show_candidate=False,\
                            show_sentence=False,\
                            show_parse=False,\
                            show_passing=False,\
                            show_correct=False,\
                            pseudo_python=False,\
                            only=[])
results

P: ('.root', ('.label', ('.bool', True), ('.bool', True)))
P: ('.root', ('.label', ('.bool', True), ('.and', ('.bool', True), ('.bool', True))))
P: ('.root', ('.label', ('.bool', True), ('.or', ('.bool', False), ('.bool', True))))
P: ('.root', ('.label', ('.bool', True), ('.not', ('.bool', False))))
P: ('.root', ('.label', ('.bool', True), ('.or', ('.bool', True), ('.and', ('.bool', True), ('.bool', False)))))
P: ('.root', ('.label', ('.bool', True), ('.call', ('.eq', ('.string', u'yes')), ('.string', u'yes'))))
P: ('.root', ('.label', ('.bool', True), ('.call', ('.eq', ('.int', 1)), ('.int', 1))))
P: ('.root', ('.label', ('.bool', True), ('.call', ('.eq', ('.int', 1)), ('.int', 1))))
P: ('.root', ('.label', ('.bool', True), ('.call', ('.lt', ('.int', 2)), ('.int', 1))))
P: ('.root', ('.label', ('.bool', True), ('.call', ('.leq', ('.int', 2)), ('.int', 2))))
P: ('.root', ('.label', ('.bool', True), ('.call', ('.gt', ('.int', 1)), ('.int', 2))))
P: ('.root', ('.label', ('.bool', True), 

,Correct,Passing,Failing,Redundant,Erroring,Unknown,Index
Example0,0,1,0,0,0,0,0
Example1,0,1,0,0,0,0,1
Example2,0,1,0,0,0,0,2
Example3,0,1,0,0,0,0,3
Example4,0,1,0,0,0,0,4
Example5,0,1,0,0,0,0,5
Example6,0,1,0,0,0,0,6
Example7,0,1,0,0,0,0,7
Example8,0,1,0,0,0,0,8
Example9,0,1,0,0,0,0,9


In [20]:
import numpy as np
print np.sum(results, axis=0)

Correct         0
Passing        60
Failing         0
Redundant       0
Erroring        0
Unknown         0
Index        1596
dtype: int64


In [21]:
(correct, passing, failing, redundant, erroring, unknown) = sp.LFs
LFs = correct + passing
print len(LFs)

60


In [22]:
from snorkel.semantic import sem_to_str

sem = ('.root', ('.label', ('.bool', True), ('.and', ('.any', ('.map', ('.in', ('.extract_text', ('.between', ('.list', ('.arg', ('.int', 1)), ('.arg', ('.int', 2)))))), ('.user_list', ('.string', u'causal')))), ('.not', ('.call', ('.in', ('.extract_text', ('.between', ('.list', ('.arg', ('.int', 1)), ('.arg', ('.int', 2)))))), ('.string', u'not'))))))
print sem
print sem_to_str(sem)

('.root', ('.label', ('.bool', True), ('.and', ('.any', ('.map', ('.in', ('.extract_text', ('.between', ('.list', ('.arg', ('.int', 1)), ('.arg', ('.int', 2)))))), ('.user_list', ('.string', u'causal')))), ('.not', ('.call', ('.in', ('.extract_text', ('.between', ('.list', ('.arg', ('.int', 1)), ('.arg', ('.int', 2)))))), ('.string', u'not'))))))
return -1 if (any(map(in text(between([arg1,arg2])), 'CAUSAL')) and not (call(in text(between([arg1,arg2])), 'not'))) else 0


In [23]:
# sp.grammar.print_chart(nested=True)

In [24]:
# sp.grammar.print_grammar()